## Le moteur de recommandation : la technique

In [ ]:
#importation des librairies et de la base de données
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

df_reco = pd.read_csv('/content/df_reco_final.csv', sep = ',') 

# On doit intégrer la librairie 'fuzzuwuzzy' pour reconnaitre le titre du film sans faute d'orthographe de l'utilisateur, 
# faire en sorte que les recommendations soient accessibles, et applicables en accédant à l'un d'entre eux,
# faire en sorte d'agrandir les images avec les infos.

#### **Le calcul de la notation des genres**
Nous avons choisi de créer un système de notation pour les genres en fonction des genres par importance. Pour cela, nous avons dû nous tourner vers la base Tmdb qui nous donne les genres des films par ordre d'importance et non ordre alphabétique comme ImdB. Cela nous permet de donner plus de poids aux genres principaux comme nous allons le voir. 

###### **classement des genres**
###### Classe 0
*   Animation 0

###### Classe 10
*   Family 10
*   Romance 12
*   Comedy 13
*   Musical 14

###### Classe 20

*   Fantasy 20
*   Mystery 21
*   Science Fiction 22

###### Classe 30
*   Adventure 30
*   Action 31
*   Western 32
*   War 33

###### Classe 40
*   Drama 40
*   Film-Noir 41
*   Thriller 42
*   Crime 43
*   Horror 44

###### Classe 50
*   Biography 50
*   History 51
*   Documentary 52

###### Classe 60
*   News 60
*   Sport 61
*   Music 62
*   Game-Show 63
*   Talk-Show 64
*   Reality-TV 65

###### **La notation**
Genre principal * 100 + genre secondaire * 10 + 3ème genre

Le genre principal est pondéré par 100 afin de regrouper tous les films du même genre principal dans les mêmes scores (ex 4000 pour les drames). Le genre secondaire est pondéré par 10 pour colorer la notation grâce à une nuance de genre importante et enfin le genre 3 qui donne une nuance supplémentaire. 



#### **Le moteur de recommendation**

In [ ]:
# création du modèle - reco sur genres uniquement sans standardisation (meilleurs résultats suite tests)

X = df_reco[['genres_notation']]
y = df_reco['fr_title']

# Model KNN (K-nearest neighbors)
# Initialization of model
distanceKNN_reco_genres_only = NearestNeighbors(n_neighbors=7).fit(X)

#### **Les résultats**

In [ ]:
# détermination des films recommandés pour "Chicken Run"
nearest = distanceKNN_reco_genres_only.kneighbors(df_reco.loc[df_reco['fr_title'] == "Chicken Run", 
                                                              ['genres_notation']])
movie_recommended = df_reco.loc[nearest[1][0]][['fr_title', 'genres_tmdb']][:7]
print(movie_recommended)

                                  fr_title                                        genres_tmdb
2014                       Peur(s) du noir  ['Animation', 'Comedy', 'Drama', 'Horror', 'My...
1087                               Hercule  ['Animation', 'Family', 'Fantasy', 'Adventure'...
2710        Astérix : Le Domaine Des Dieux     ['Animation', 'Comedy', 'Adventure', 'Family']
584                     Le Roi et l’oiseau                 ['Animation', 'Family', 'Fantasy']
2533                             Ballerina     ['Animation', 'Adventure', 'Comedy', 'Family']
2071  Epic : La bataille du royaume secret    ['Animation', 'Adventure', 'Family', 'Fantasy']
2724           Mune, Le gardien de la Lune  ['Animation', 'Family', 'Adventure', 'Fantasy'...


In [ ]:
# détermination des films recommandés pour "Intouchables"
nearest = distanceKNN_reco_genres_only.kneighbors(df_reco.loc[df_reco['fr_title'] == "Intouchables", 
                                                              ['genres_notation']])
movie_recommended = df_reco.loc[nearest[1][0]][['fr_title', 'genres_tmdb']][:7]
print(movie_recommended)

                  fr_title          genres_tmdb
375         La voie lactée  ['Drama', 'Comedy']
755      Ombres au paradis  ['Drama', 'Comedy']
223  Séduite et abandonnée  ['Drama', 'Comedy']
343                 A tanú  ['Drama', 'Comedy']
99       Une leçon d'amour  ['Drama', 'Comedy']
440       La Grande Bouffe  ['Drama', 'Comedy']
186            Le Fanfaron  ['Drama', 'Comedy']


In [ ]:
# détermination des films recommandés pour "Le fabuleux destin d'Amélie Poulain"
nearest = distanceKNN_reco_genres_only.kneighbors(df_reco.loc[df_reco['fr_title'] == "Le Fabuleux Destin d'Amélie Poulain", 
                                                              ['genres_notation']])
movie_recommended = df_reco.loc[nearest[1][0]][['fr_title', 'genres_tmdb']][:7]
print(movie_recommended)

                                   fr_title            genres_tmdb
998                            Gazon maudit  ['Comedy', 'Romance']
897                         Ми нисмо анђели  ['Comedy', 'Romance']
275                   La Mégère apprivoisée  ['Comedy', 'Romance']
963              Tiens ton foulard, Tatiana  ['Comedy', 'Romance']
113               Sourires d'une nuit d'été  ['Comedy', 'Romance']
406                                Avanti !  ['Comedy', 'Romance']
1388  Comment se faire larguer en 10 leçons  ['Comedy', 'Romance']


In [ ]:
# détermination des films recommandés pour "Fight Club" 
nearest = distanceKNN_reco_genres_only.kneighbors(df_reco.loc[df_reco['fr_title'] == "Fight Club", 
                                                              ['genres_notation']])
movie_recommended = df_reco.loc[nearest[1][0]][['fr_title', 'genres_tmdb', 'genres_notation']][:7]

print(movie_recommended)

                          fr_title genres_tmdb  genres_notation
9            Le dernier des hommes   ['Drama']             4000
124           Les Nuits de Cabiria   ['Drama']             4000
21   Le Journal d'une fille perdue   ['Drama']             4000
128                      Le Visage   ['Drama']             4000
28       Jeunes Filles en uniforme   ['Drama']             4000
22                     L'Ange bleu   ['Drama']             4000
47                       Tourments   ['Drama']             4000


## Les pistes d'amélioration

###### **Le critère pour la détermination du pays d'un film**
Les possibilités
*   pays de production principal
*   pays de production si co-production sans prise en compte de la participation des pays co-producteurs (ce que proposent ImdB et Tmdb et qui est repris ici)
*   pays de production si co-production avec prise en compte d'une limite minimale de participation des pays co-producteurs (peut permettre au public de se rendre compte des productions que financent les pays européens)
*   le pays de la propriété intellectuelle du film

###### **Affinage de la notation des genres**
*   3 ou 4 genres par film au minimum (actuellement certains films ne comptent qu'1 ou 2 genre(s), ce qui nuit à la pertinence du moteur)
*   Tests de plus grande importance pour réviser le regroupement des genres par classe si besoin et réviser la notation choisie

###### **Inclusion d'autres paramètres**
*   Les acteurs "têtes d'affiche" : possibilité de créer un système de notation similaire à celui des genres pour les 15/20 acteurs pouvant attirer le public 
*   Une même logique pourrait être appliquée pour les directeurs
*   Pour les acteurs et les directeurs, on pourrait imaginer que le téléspectateur puisse choisir les reco selon ces critères (acteur choisi voire acteurs similaires, idem pour directeur)
*   Si toutes ces améliorations sont faites, on peut également penser à tester l'ajout des critères de rating et de votes (nuisaient à la performance dans un 1er temps mais pourrait apporter un plus avec le moteur reconfiguré)




